# Data processing for the Organisational Audit Portfolio spreadsheet 2: rename and reduce data

This notebook turns the MultiIndex column and row names into a single name and saves a lookup table of the new names to the old full data.

In [1]:
import os
import pandas as pd

### Import data

In [2]:
dir_files = '../data/organisational_audit/processed'
file_excel = 'processed_2019_portfolio_key_indicators_summary_multiindex.csv'

In [3]:
df = pd.read_csv(
    os.path.join(dir_files, file_excel),
    header=[0, 1, 2, 3, 4, 5],
    index_col=[0, 1, 2, 3]
)

In [4]:
df.columns.names

FrozenList(['SCN', 'Trust name', 'Site name', 'Hospital names', 'hospital_name_2', 'hospital_name_3'])

In [5]:
df.index.names

FrozenList(['Key indicator', 'Response required to meet indicator', 'key_indicator_group', 'question'])

### Header: Check for redundant site names

In [6]:
print(f'Total hospitals: {len(df.columns)}')

for header in df.columns.names:
    n_unique = len(df.columns.get_level_values(header).unique())
    print(f'{header}: {n_unique}')

Total hospitals: 172
SCN: 16
Trust name: 138
Site name: 172
Hospital names: 172
hospital_name_2: 172
hospital_name_3: 172


All hospitals have a unique site name, so we'll move the 'site name' data into another dataframe.

### Header: reduce hospital names

Re-jig "hospital names" column header. In the Excel sheet it is a merged cell and applies to three rows of data. The hospital can have multiple names in English and in Welsh so all should be kept.

Only keep the top row in this DataFrame. Store the other rows' data in a new DataFrame for reference.

Also move the 'site names' data to this new DataFrame.

In [7]:
headers_to_move = ['hospital_name_2', 'hospital_name_3', 'SCN', 'Trust name', 'Site name']

In [8]:
# Combine those rows into one dataframe:
df_hospital_names = pd.DataFrame(
    [df.columns.get_level_values(h) for h in ['Hospital names'] + headers_to_move],
    index=['hospital_name_1', 'hospital_name_2', 'hospital_name_3', 'scn', 'trust', 'site_name']
).T

In [9]:
df_hospital_names.head(3)

,hospital_name_1,hospital_name_2,hospital_name_3,scn,trust,site_name
0,Unnamed: 4_level_3,Unnamed: 4_level_4,Unnamed: 4_level_5,National Results,Unnamed: 4_level_1,Unnamed: 4_level_2
1,Queens Hospital Romford HASU,Queens Hospital Romford SU,Unnamed: 5_level_5,London,"Barking, Havering and Redbridge University Hos...","Barking, Havering and Redbridge University Hos..."
2,Newham General Hospital,Unnamed: 6_level_4,Unnamed: 6_level_5,London,Barts Health NHS Trust,Barts Health NHS Trust (Newham University Hosp...


In [10]:
# Remove the 'National Results' row:
# (assuming it's first in the list)
df_hospital_names = df_hospital_names.iloc[1:]

In [11]:
for col in df_hospital_names.columns:
    mask = df_hospital_names[col].str.startswith('Unnamed')
    df_hospital_names.loc[mask, col] = ''

Save hospital names df to file:

In [12]:
df_hospital_names.to_csv(os.path.join(dir_files, 'hospital_names_trusts.csv'), index=False)

Example of how to access a hospital if you know one of its names but not which column of the DataFrame that name is in:

In [13]:
def find_main_hospital_name(
        df_hospital_names: 'pd.DataFrame',
        name_to_look_up: 'str',
        column_main_name: 'str'='hospital_name_1'
        ):
    # df of True/False for name matches the name_to_look_up:
    df_bool = df_hospital_names.eq(name_to_look_up)
    # series of True/False, one for each row in the dataframe,
    # and the row is True when any value in the row in df_bool is True.
    series_bool = df_bool.any(axis='columns')
    # Use that series as a mask to pick out only the right row,
    # then pick out the value in that row and the right column:
    main_hospital_name = df_hospital_names.loc[series_bool, column_main_name].values[0]
    return main_hospital_name

In [14]:
name_to_look_up = 'Northwick Park Hospital SU'

find_main_hospital_name(df_hospital_names, name_to_look_up)

'Northwick Park Hospital HASU'

Drop the other hospital name headers from the organisational audit dataframe:

In [15]:
df.columns = df.columns.droplevel(headers_to_move)

Rename the 'National results' column:

In [16]:
col = df.columns[df.columns.str.startswith('Unnamed')].values[0]

df = df.rename(columns={col: 'National Results'})

In [17]:
df.head(3)

Hospital names                                                                                                                                                       National Results  \
Key indicator                                      Response required to meet indicator                key_indicator_group question                                                      
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN       1: 5% (8/169)_x000D_\n2: 7% (12/169)_x000D_\n3...   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                                            58% (98/169)   
                                                   Band 6 nurses WTE per 10 beds                      Staffing/Workforce  NaN                      1.9 (1.4-2.9)_x000D_\nMedian (IQR)   

Hospital names                                                                                                                     Queens Hospital Romford HASU  \
Key indicator                                      Response required to meet indicator                key_indicator_group question                                
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN                                 6   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                               Yes   
                                                   Band 6 nurses WTE per 10 beds                      Staffing/Workforce  NaN                              2.73   

Hospital names                                                                                                                     Newham General Hospital  \
Key indicator                                      Response required to meet indicator                key_indicator_group question                           
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN                            6   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                          Yes   
                                                   Band 6 nurses WTE per 10 beds                      Staffing/Workforce  NaN                         1.54   

Hospital names                                                                                                                     Royal London Hospital HASU  \
Key indicator                                      Response required to meet indicator                key_indicator_group question                              
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN                               5   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                             Yes   
                                                   Band 6 nurses WTE per 10 beds                      Staffing/Workforce  NaN                            2.85   

Hospital names                                                                                                                     Whipps Cross University Hospital  \
Key indicator                                      Response required to meet indicator                key_indicator_group question                                    
Total Key Indicators Achieved \n(Post 72hrs sit... NaN                                                NaN                 NaN                                     3   
1                                                  Key Indicator 1:  Minimum establishment of band... Staffing/Workforce  NaN                                   

## Set up new index names

Make a new set of data that links the new short names with the full names they were once called and any other notes.

Copy the existing index, make a new column for the short name, and save to file.

In the final dataframe, only keep the short names.

Convert the index to its own Dataframe to have a better look at it:

In [18]:
df_index = df.index.to_frame(index=False)

df_index['property'] = ''

df_index = df_index.fillna('')
df_index = df_index.astype(object)

df_index

,Key indicator,Response required to meet indicator,key_indicator_group,question,property
0,Total Key Indicators Achieved \n(Post 72hrs si...,,,,
1,1,Key Indicator 1: Minimum establishment of ban...,Staffing/Workforce,,
2,1,Band 6 nurses WTE per 10 beds,Staffing/Workforce,,
3,1,Band 7 nurses WTE per 10 beds,Staffing/Workforce,,
4,2,Key Indicator 2: Presence of a clinical psycho...,Staffing/Workforce,,
5,2,Clinical psychologist WTE per 30 beds (qualified),Staffing/Workforce,,
6,3,Key Indicator 3: Out of hours presence of str...,7-day working,,
7,3,Out of hours,7-day working,1.7. Do you have stroke specialist nurses (ban...,
8,3,Registered nurses Type 1 Beds (weekdays 10 pm),7-day working,1.7. Do you have stroke specialist nurses (ban...,
9,3,Registered nurses Type 1 Beds (Saturdays),7-day working,1.7. Do you have stroke specialist nurses (ban...,


In [19]:
# Name the main "key indicator" rows:
ki_str = [f'Key Indicator {i}' for i in range(1, 11, 1)]
rename_str = [f'ki{i}' for i in range(1, 11, 1)]

index_names_ki = dict(zip(ki_str, rename_str))

index_names_ki

{'Key Indicator 1': 'ki1',
 'Key Indicator 2': 'ki2',
 'Key Indicator 3': 'ki3',
 'Key Indicator 4': 'ki4',
 'Key Indicator 5': 'ki5',
 'Key Indicator 6': 'ki6',
 'Key Indicator 7': 'ki7',
 'Key Indicator 8': 'ki8',
 'Key Indicator 9': 'ki9',
 'Key Indicator 10': 'ki10'}

In [20]:
rename_dict = {
    'Total Key Indicators Achieved \n(Post 72hrs sites receive points from KIs 3,4, and 6  from their main acute site)': 'ki_total',
    # Key indicator 1
    'Band 6 nurses WTE per 10 beds': 'ki1_nurses6_wte',
    'Band 7 nurses WTE per 10 beds': 'ki1_nurses7_wte',
    # Key indicator 2
    'Clinical psychologist WTE per 30 beds (qualified)': 'ki2_psych_wte',
    # Key indicator 3
    'Out of hours': 'ki3_strokenurse_outofhours',
    'Registered nurses Type 1 Beds (weekdays 10 pm)': 'ki3_strokenurse_bed1_weekdays10pm',
    'Registered nurses Type 1 Beds (Saturdays)': 'ki3_strokenurse_bed1_saturdays',
    'Registered nurses Type 1 Beds (Saturdays 10 pm)': 'ki3_strokenurse_bed1_saturdays10pm',
    'Registered nurses Type 1 Beds (Sundays)': 'ki3_strokenurse_bed1_sundays',
    'Registered nurses Type 1 Beds (Sundays 10 pm)': 'ki3_strokenurse_bed1_sundays10pm',
    'Registered nurses Type 3 Beds (weekdays 10 pm)': 'ki3_strokenurse_bed3_weekdays10pm',
    'Registered nurses Type 3 Beds (Saturdays)': 'ki3_strokenurse_bed3_saturdays',
    'Registered nurses Type 3 Beds (Saturdays 10 pm)': 'ki3_strokenurse_bed3_saturdays10pm',
    'Registered nurses Type 3 Beds (Sundays)': 'ki3_strokenurse_bed3_sundays',
    'Registered nurses Type 3 Beds (Sundays 10 pm)': 'ki3_strokenurse_bed3_sundays10pm',
    # Key indicator 4
    'Nurses per 10 type 1 beds 10am Saturdays': 'ki4_minimum_nurse_bed1_saturdays10am',
    'Nurses per 10 type 1 beds 10am Sundays': 'ki4_minimum_nurse_bed1_sundays10am',
    'Nurses per 10 type 3 beds 10am Saturdays\n': 'ki4_minimum_nurse_bed3_saturdays10am',
    'Nurses per 10 type 3 beds 10am Sundays\n': 'ki4_minimum_nurse_bed3_sundays10am',
    # Key indicator 5
    '(e) Occupational Therapy (qualified)': 'ki5_occupationaltherapy_7days',
    '(g) Physiotherapy (qualified)': 'ki5_physiotherapy_7days',
    '(i) Speech & Language Therapy (qualified)': 'ki5_speechlanguagetherapy_7days',
    # Key indicator 6
    'Thrombolysis candidates only': 'ki6_prealert_ivtcandidates',
    'All FAST positive': 'ki6_prealert_fastpositive',
    'All other suspected strokes': 'ki6_prealert_othersuspectedstroke',
    'Stroke Specialist Nurse': 'ki6_prealert_strokenurse',
    'Stroke Junior Doctor on call': 'ki6_prealert_juniordoctor',
    'Stroke Consultant on call': 'ki6_prealert_consultant',
    # Key indicator 7
    '6.1. Do you have access to at least one stroke/neurology specific early supported discharge multidisciplinary team?': 'ki7_stroke_esd_team',
    '6.1 (b) What percentage of your patients has access to at least one of these teams if needed?': 'ki7_percent_access_to_stroke_esd_team',
    # Key indicator 8
    '9.7. How often is there a formal survey seeking patient/carer views on stroke services? (This does not include the Friends and Family Test)': 'ki8_patientcarersurvey_frequency',
    # Key indicator 9
    '(a) First line brain imaging': 'ki9_tia_brainimaging_mostused',
    'Outpatient': 'ki9_tia_outpatients_seen',
    'Outpatient timescale': 'ki9_tia_outpatient_timescale',
    # Key indicator 10
    'Executive on the Board': 'ki10_management_executiveonboard',
    'Non-executive on the Board': 'ki10_management_nonexecutiveonboard',
    'Chairman of Clinical Governance (or equivalent)': 'ki10_management_chairmanofclinicalgovernance',
}


In [21]:
# Lazy multiple loops to hit all of the new name locations:

for k, v in index_names_ki.items():
    mask = df_index['Response required to meet indicator'].str.startswith(k)
    df_index.loc[mask, 'property'] = v

for k, v in rename_dict.items():
    mask = df_index['Key indicator'] == k
    df_index.loc[mask, 'property'] = v

for k, v in rename_dict.items():
    mask = df_index['Response required to meet indicator'] == k
    df_index.loc[mask, 'property'] = v

In [22]:
df_index

,Key indicator,Response required to meet indicator,key_indicator_group,question,property
0,Total Key Indicators Achieved \n(Post 72hrs si...,,,,ki_total
1,1,Key Indicator 1: Minimum establishment of ban...,Staffing/Workforce,,ki1
2,1,Band 6 nurses WTE per 10 beds,Staffing/Workforce,,ki1_nurses6_wte
3,1,Band 7 nurses WTE per 10 beds,Staffing/Workforce,,ki1_nurses7_wte
4,2,Key Indicator 2: Presence of a clinical psycho...,Staffing/Workforce,,ki2
5,2,Clinical psychologist WTE per 30 beds (qualified),Staffing/Workforce,,ki2_psych_wte
6,3,Key Indicator 3: Out of hours presence of str...,7-day working,,ki3
7,3,Out of hours,7-day working,1.7. Do you have stroke specialist nurses (ban...,ki3_strokenurse_outofhours
8,3,Registered nurses Type 1 Beds (weekdays 10 pm),7-day working,1.7. Do you have stroke specialist nurses (ban...,ki3_strokenurse_bed1_weekdays10pm
9,3,Registered nurses Type 1 Beds (Saturdays),7-day working,1.7. Do you have stroke specialist nurses (ban...,ki3_strokenurse_bed1_saturdays


Save a copy of this new index dataframe:

In [23]:
df_index.to_csv(os.path.join(dir_files, 'key_indicator_lookup.csv'), index=False)

### Update main dataframe with new index column

In [24]:
# Replace the old index with the new column:
df.index = df_index['property']

In [25]:
df

Hospital names,National Results,Queens Hospital Romford HASU,Newham General Hospital,Royal London Hospital HASU,Whipps Cross University Hospital,Charing Cross Hospital HASU,King's College Hospital HASU,Princess Royal University Hospital HASU,Northwick Park Hospital HASU,St George's Hospital HASU,...,Causeway Hospital,Antrim Area Hospital,Ulster Hospital,Craigavon Area Hospital,Daisy Hill Hospital,Altnagelvin Hospital,South West Acute Hospital,Noble's Hospital,Walton Centre Stroke Team,Queen's Medical Centre - Nottingham
property,,,,,,,,,,,,,,,,,,,,,
ki_total,1: 5% (8/169)_x000D_\n2: 7% (12/169)_x000D_\n3...,6,6,5,3,5,9,7,9,6,...,2,2,5,2,2,3,3,3,NaN,NaN
ki1,58% (98/169),Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,No,No,No,Yes,NaN,NaN
ki1_nurses6_wte,1.9 (1.4-2.9)_x000D_\nMedian (IQR),2.73,1.54,2.85,2.63,3.67,3.21,3.25,4.2,3.33,...,1.25,2.33,2.08,2.63,1.5,1.2,1.56,5,NaN,NaN
ki1_nurses7_wte,0.5 (0.4-0.8)_x000D_\nMedian (IQR),1.14,1.54,0.38,0.53,0.44,0.71,1.25,0.6,1.11,...,1.08,1.17,0.42,0.26,0.33,0.4,0.56,0.83,NaN,NaN
ki2,7% (12/169),No,No,No,No,No,Yes,No,Yes,No,...,No,No,No,No,No,No,No,No,NaN,NaN
ki2_psych_wte,0.1 (0-0.3)_x000D_\nMedian (IQR),0.48,0,0.23,0,0.27,1.29,0.75,1.14,0.83,...,0,0,0,0,0,0,0,0,NaN,NaN
ki3,71% (101/142),Yes,At site treating your patients during the firs...,Yes,No,Yes,Yes,Yes,Yes,Yes,...,No,No,Yes,No,No,No,No,Yes,NaN,NaN
ki3_strokenurse_outofhours,71% (101/142),Yes,At site treating your patients during the firs...,Yes,No,Yes,Yes,Yes,Yes,Yes,...,No,No,Yes,No,No,No,No,Yes,NaN,NaN
ki3_strokenurse_bed1_weekdays10pm,215,5,Not Applicable,4,Not Applicable,6,5,6,7,8,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,1,1,Not Applicable,NaN,NaN


### Split off 'National Results'

This column is formatted too differently from the individual hospitals to keep it in with the rest.

In [26]:
df_national_results = df['National Results']

In [27]:
df_national_results.head(3)

property
ki_total           1: 5% (8/169)_x000D_\n2: 7% (12/169)_x000D_\n3...
ki1                                                     58% (98/169)
ki1_nurses6_wte                   1.9 (1.4-2.9)_x000D_\nMedian (IQR)
Name: National Results, dtype: object

Drop the national results from the main organisational audit dataframe:

In [29]:
df = df.drop('National Results', axis='columns')

In [30]:
df.head(3)

Hospital names,Queens Hospital Romford HASU,Newham General Hospital,Royal London Hospital HASU,Whipps Cross University Hospital,Charing Cross Hospital HASU,King's College Hospital HASU,Princess Royal University Hospital HASU,Northwick Park Hospital HASU,St George's Hospital HASU,University College Hospital HASU,...,Causeway Hospital,Antrim Area Hospital,Ulster Hospital,Craigavon Area Hospital,Daisy Hill Hospital,Altnagelvin Hospital,South West Acute Hospital,Noble's Hospital,Walton Centre Stroke Team,Queen's Medical Centre - Nottingham
property,,,,,,,,,,,,,,,,,,,,,
ki_total,6,6,5,3,5,9,7,9,6,7,...,2,2,5,2,2,3,3,3,NaN,NaN
ki1,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,No,No,No,Yes,NaN,NaN
ki1_nurses6_wte,2.73,1.54,2.85,2.63,3.67,3.21,3.25,4.2,3.33,5.71,...,1.25,2.33,2.08,2.63,1.5,1.2,1.56,5,NaN,NaN


## Save results to file

Also at this point, transpose the data.

In [34]:
df = df.T

In [35]:
# Takes a bit more persuading to transpose a Series:
df_national_results = pd.DataFrame(df_national_results).T

In [37]:
df.to_csv(os.path.join(dir_files, 'processed_2019_portfolio_key_indicators_summary.csv'))
df_national_results.to_csv(os.path.join(dir_files, 'processed_2019_portfolio_key_indicators_summary_national.csv'))